In [21]:
!pip install transformers
!pip install IPython
!pip install datasets
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
from datasets import load_dataset

dataset = load_dataset("librispeech_asr",
                       split="train.clean.100",
                       streaming=True,
                       trust_remote_code=True)

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

librispeech_asr.py:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

In [4]:
example = next(iter(dataset))

dict_keys(['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'])


In [5]:
print(example)

{'file': '374-180298-0000.flac', 'audio': {'path': '374-180298-0000.flac', 'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
       -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]), 'sampling_rate': 16000}, 'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED', 'speaker_id': 374, 'chapter_id': 180298, 'id': '374-180298-0000'}


In [7]:
print(example['audio']['sampling_rate'])

16000


# Display the audio sample

In [10]:
from IPython.display import Audio
Audio(example['audio']['array']
      ,rate=example['audio']['sampling_rate'])

In [12]:
example2 = dataset.take(5)
# print(example2)

In [14]:
print(list(example2)[2])

{'file': '374-180298-0002.flac', 'audio': {'path': '374-180298-0002.flac', 'array': array([-2.44140625e-04, -2.44140625e-04, -1.83105469e-04, ...,
        1.83105469e-04,  3.05175781e-05, -1.52587891e-04]), 'sampling_rate': 16000}, 'text': 'I WISHED ABOVE ALL NOT TO LEAVE MYSELF TIME TO THINK OVER THE POSITION I HAD ACCEPTED FOR IN SPITE OF MYSELF IT WAS A GREAT DISTRESS TO ME THUS MY LIFE GENERALLY SO CALM', 'speaker_id': 374, 'chapter_id': 180298, 'id': '374-180298-0002'}


In [15]:
Audio(list(example2)[2]['audio']['array']
      ,rate=list(example2)[2]['audio']['sampling_rate'])

# let build voice recognizer that can take our voice and actualy give text output

In [16]:
from transformers import pipeline

asr = pipeline(task="automatic-speech-recognition",
               model="distil-whisper/distil-small.en")

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/332M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cpu


In [17]:
asr.feature_extractor.sampling_rate

16000

In [18]:
example['audio']['sampling_rate'] #you need to check the compatiblity of the sampling rate of the example to model

16000

In [19]:
example["text"]

'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED'

In [22]:
import os
import gradio as gr

In [23]:
demo = gr.Blocks()

In [24]:
def transcribe_speech(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(filepath)
    return output["text"]


In [34]:
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never")


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [26]:
file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never",
)


In [36]:
import gradio as gr
import os

def transcribe_speech(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(filepath)
    return output["text"]

with gr.Blocks() as demo:
    with gr.Tabs():
        with gr.Tab("Transcribe Microphone"):
            gr.Interface(
                fn=transcribe_speech,
                inputs=gr.Audio(sources="microphone", type="filepath"),
                outputs=gr.Textbox(label="Transcription", lines=3),
                allow_flagging="never"
            ).render()

        with gr.Tab("Transcribe Audio File"):
            gr.Interface(
                fn=transcribe_speech,
                inputs=gr.Audio(sources="upload", type="filepath"),
                outputs=gr.Textbox(label="Transcription", lines=3),
                allow_flagging="never"
            ).render()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


## It Work for the upload file but it doesn't work for the mic i think it is b/c i am in the olline lab ... it should work if it is run on local computer

In [ ]:
demo.launch(share=True, server_port=int(os.environ.get('PORT1', 8000)))

In [46]:
demo.close()

Closing server running on port: 8000


In [ ]:
# the issue is it does n't work for more longer audio fies

# to make the model take more tha 30 sec voice

In [45]:
#update the code
def transcribe_long_form(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(
      filepath,
      max_new_tokens=256,
      chunk_length_s=30,
      batch_size=8,
    )
    return output["text"]

In [47]:
demo.launch(share=True, server_port=int(os.environ.get('PORT1', 8001)))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f8be234edcfbf48663.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 